#######################################################################################################

Executes daly to update all the stock signals 
Current signals -
1. EMA Cross indicator
2. Turn Around Stocks

#######################################################################################################


In [1]:
import os, sys

# Get repo root (works both locally & on GitHub Actions)
repo_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
src_path = os.path.join(repo_root, "src")

if src_path not in sys.path:
    sys.path.append(src_path)

print("✅ Added to path:", src_path)
print("Current working directory:", os.getcwd())

# Now your imports will work
import get_stocks_data as gs

# from src import __version__
# print(__version__)

✅ Added to path: d:\Work\Projects\Stock_Market\Stock_Market\Stock_signals\Stock-Signals\src
Current working directory: d:\Work\Projects\Stock_Market\Stock_Market\Stock_signals\Stock-Signals\notebooks


In [2]:
import numpy as np 
import pandas as pd 
import yfinance as yf
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',120)
import datetime as dt
import datetime
import time 
import ta
import sqlite3

In [4]:
analysis_period = 30 # past 30 days for Analysis
raw_data_needed = analysis_period + 2 * 365  # +1 year of buffer for calculating indicators
output_period = 30 # restrict output file for EMA-signals in 30 days

In [5]:
# Determine repository root
# GitHub sets GITHUB_WORKSPACE; locally, we use current working directory
repo_root = os.environ.get("GITHUB_WORKSPACE", os.getcwd())

# if repo_root == os.getcwd(): #then go one directory above to reach repo root
#     repo_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

if "d:\\Work" in repo_root or "D:\\Work" in repo_root:
    repo_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

print("Repository root determined as:", repo_root)

# Construct full path to the database
db_path = os.path.join(repo_root, "data", "stocks.db")

# Optional: check if it exists
if not os.path.exists(db_path):
    print(f"ERROR: Database not found at {db_path}")
    sys.exit(1)  # stop execution if DB missing

print("Using database at:", db_path)

# Now you can safely call your function
# Example:
# stocklist = gs.get_company_symbols(db_path, 'COMPANY_DETAILS')


Repository root determined as: d:\Work\Projects\Stock_Market\Stock_Market\Stock_signals\Stock-Signals
Using database at: d:\Work\Projects\Stock_Market\Stock_Market\Stock_signals\Stock-Signals\data\stocks.db


In [6]:

stocklist = gs.get_company_symbols(db_path, 'COMPANY_DETAILS')
stocklist.head()

tickers = list(stocklist.ticker.unique())
print(len(tickers))
# tickers = ["ESSARSHPNG.NS","LATENTVIEW.NS","BHEL.NS"]

stock_df = gs.fetch_stock_data(tickers, raw_data_needed)

stock_df.head()

2025-11-10 08:44:13,856 - INFO - 📈 Fetching data for 2178 tickers...
2025-11-10 08:44:13,859 - INFO - Fetching data from 2023-10-12 to 2025-11-10


d:\Work\Projects\Stock_Market\Stock_Market\Stock_signals\Stock-Signals\data\stocks.db
2178


[*********************100%***********************]  2178 of 2178 completed
2025-11-10 08:45:44,599 - ERROR - 
1 Failed download:
2025-11-10 08:45:44,600 - ERROR - ['NDTV-RE.NS']: YFInvalidPeriodError("NDTV-RE.NS: Period 'max' is invalid, must be of the format 1d, 5d, etc.")
2025-11-10 08:45:49,353 - INFO - ✅ Download completed in 95.51 seconds
2025-11-10 08:45:53,981 - INFO - 📊 Final dataset: 1,029,075 rows from 2023-10-12 to 2025-11-07


Price,Date,Ticker,Adj Close,Close,High,Low,Open,Volume,20ema,50ema,100ema,200ema,30dv,7dv,Symbol
0,2023-10-12,20MICRONS.NS,NaN,144.176346,145.166228,144.027870,144.176346,66373.0,144.176346,144.176346,144.176346,144.176346,NaN,NaN,20MICRONS
1,2023-10-13,20MICRONS.NS,NaN,141.305695,144.077368,141.305695,141.305695,29996.0,143.902950,144.063771,144.119501,144.147782,NaN,NaN,20MICRONS
2,2023-10-16,20MICRONS.NS,NaN,138.484528,138.484528,138.484528,138.484528,40521.0,143.386910,143.844977,144.007918,144.091431,NaN,NaN,20MICRONS
3,2023-10-17,20MICRONS.NS,NaN,141.206696,141.206696,138.484519,141.206696,34376.0,143.179271,143.741515,143.952448,144.062728,NaN,NaN,20MICRONS
4,2023-10-18,20MICRONS.NS,NaN,138.385544,143.136982,138.385544,138.385544,20591.0,142.722725,143.531477,143.842212,144.006238,NaN,NaN,20MICRONS


##### Call Accumulation signal

In [7]:
import importlib
import accumulation_signal as acs
importlib.reload(acs)

# Detect accumulation
accumulation_df = acs.detect_gradual_accumulation(stock_df)

# Filter the ones meeting entry criteria
accum_df = acs.filter_accumulation(accumulation_df)


status  = acs.save_accumulation_to_db(accum_df, db_path)
if status:
    print("✅ Database updated successfully")
else:
    print("⚠️ Database Update failed")
# View
accum_df.head()

✅ Database updated successfully


,Symbol,Date,AD_Slope,Above_20EMA_%,Avg_Volume_Spike,RSI,Accumulating,20ema,50ema,200ema,Close,7dv,30dv
0,GLOBE,2025-11-07,106228.7604,80.0,1.43,68.49,True,2.403263,2.319979,2.695159,2.510000,2.303447e+06,1.105821e+06
1,RBZJEWEL,2025-11-07,1155.2315,80.0,1.41,65.00,True,146.405586,142.920478,147.095673,156.350006,3.652081e+05,1.561898e+05
2,ASHOKA,2025-11-07,124864.8468,90.0,1.30,58.05,True,196.244525,193.496737,201.960307,198.919998,1.577122e+06,8.100797e+05
3,ASTRAL,2025-11-07,267446.4446,95.0,1.26,73.01,True,1465.859318,1443.114895,1487.197912,1557.300049,1.247884e+06,7.025423e+05
4,REPCOHOME,2025-11-07,34139.5674,100.0,1.07,62.96,True,412.542336,396.749951,397.905690,425.600006,2.851806e+05,1.827009e+05


#### EMA Signal indicator 


In [18]:
import importlib
import ema_signal as emasignal
importlib.reload(emasignal)

# Detect Golden/Death cross with volume confirmation

long_stocks, short_stocks, ema_df = emasignal.generate_ema_signals(
    stock_df,
    analysis_period=15,
    vol_thresh=1.5,
    output_period=30
)
long_stocks.head()



status  = emasignal.save_EMA_signal_to_db(long_stocks, db_path)
if status:
    print("✅ Database updated successfully")
else:
    print("⚠️ Database Update failed")
# View
ema_df.head()

178 tickers with Close/200ema crossover between 2025-10-26–2025-11-10
38 tickers with 50ema/200ema crossover between 2025-10-26–2025-11-10
✅ Metadata fetched for 205 total stocks
✅ Final signals: 138 long | 82 short
✅ Saved 138 new EMA signal records to SIGNAL_EMA_CROSS
📦 Table now has 3609 total records.
✅ Database updated successfully


,Symbol,signal_1,Date_1,Price_1,7dratio_1,30dratio_1,Vol1,Mom1,signal_2,Date_2,Price_2,7dratio_2,30dratio_2,Vol2,Mom2,currentPrice,Capitalization,MCap,upFrom52wlow,PE,PB,PBXPE,debtToEquity,PS,PricetoCash,EPS,CashperShare,update date,beta,AD,AD_50
0,3MINDIA,Price_gt_200ema,2025-11-03,30700.0,4.29,1.55,4.29,-2.0,Golden cross,2025-11-04,35885.0,5.91,3.38,5.91,29.0,35440.0,Large Cap,39923.0,0.86,72.0,25.0,1789.0,0.06,8.40,64.448785,491.0,550.0,2025-11-10,-0.05,0.69,0.98
1,A2ZINFRA,Price_gt_200ema,2025-11-07,18.0,2.38,1.35,2.38,-1.0,,,NaN,NaN,NaN,NaN,NaN,18.0,Nano Cap,322.0,0.41,33.0,7.0,235.0,3.45,0.94,66.545455,1.0,0.0,2025-11-10,0.17,NaN,NaN
2,AARVEEDEN,Price_gt_200ema,2025-11-06,143.0,0.79,2.14,2.14,-24.0,,,NaN,NaN,NaN,NaN,NaN,151.0,Nano Cap,390.0,0.71,19.0,10.0,184.0,2.64,13.75,26.197232,8.0,6.0,2025-11-10,0.34,NaN,NaN
3,ABBOTINDIA,,,NaN,NaN,NaN,NaN,NaN,Death cross,2025-10-31,28995.0,1.95,0.86,1.95,-33.0,28790.0,Large Cap,61641.0,0.30,42.0,15.0,644.0,0.05,9.16,47.824238,682.0,602.0,2025-11-10,0.00,-0.77,-0.51
4,ADANIENT,Price_gt_200ema,2025-10-29,2537.0,2.22,0.47,2.22,20.0,,,NaN,NaN,NaN,NaN,NaN,2375.0,Large Cap,287057.0,0.38,36.0,5.0,189.0,1.78,3.09,22.711884,67.0,105.0,2025-11-10,0.55,NaN,NaN


##### Update the Stock returns table

In [35]:
# Update the signal returns 
# import accumulation_signal as acs

import importlib
import signal_returns as sigret
importlib.reload(sigret)

# from signal_returns import generate_signal_returns

status, final_returns = sigret.generate_signal_returns(db_path, stock_df)

if status:
    print("✅ Signal returns generation successful!")
    display(final_returns.head())
else:
    print("❌ Signal returns generation failed.")


✅ Total signals loaded: 1906
🔄 Corporate action adjustments applied to 1724 signals
       Symbol Signal_date          Signal_Type  Signal_Price  Signal_Rank  \
0       CELLO  2025-11-07  Accumulation Signal    635.299988            1   
1  KEYFINSERV  2025-11-07         Golden cross    342.000000            1   
2       GLOBE  2025-11-07  Accumulation Signal      2.510000            2   
3      SOLARA  2025-11-07          Death cross    565.000000            2   
4    RBZJEWEL  2025-11-07  Accumulation Signal    156.350006            3   

   Updated_Signal_Price  
0            635.299988  
1            341.549988  
2              2.510000  
3            565.049988  
4            156.350006  
✅ Total signals after merging: 322185
📦 Existing SIGNAL_RETURNS records: 2173
✅ New unique records found: 0
💾 SIGNAL_RETURNS table updated successfully.
🔒 Database connection closed.
✅ Signal returns generation successful!


,Symbol,Signal_Type,Signal_date,Signal_Price,Signal_Rank,ret_1w,ret_1w_price,ret_1w_date,ret_2w,ret_2w_price,ret_2w_date,ret_1m,ret_1m_price,ret_1m_date,ret_3m,ret_3m_price,ret_3m_date,ret_6m,ret_6m_price,ret_6m_date,ret_1y,ret_1y_price,ret_1y_date,ret_sinceSignal,current_price,current_date,ret_1w_perc,ret_2w_perc,ret_1m_perc,ret_3m_perc,ret_6m_perc,ret_1y_perc,ret_sinceSignal_perc


Price,Date,Ticker,Adj Close,Close,High,Low,Open,Volume,20ema,50ema,100ema,200ema,30dv,7dv,Symbol
0,2023-10-12,20MICRONS.NS,NaN,144.176346,145.166228,144.027870,144.176346,66373.0,144.176346,144.176346,144.176346,144.176346,NaN,NaN,20MICRONS
1,2023-10-13,20MICRONS.NS,NaN,141.305695,144.077368,141.305695,141.305695,29996.0,143.902950,144.063771,144.119501,144.147782,NaN,NaN,20MICRONS
2,2023-10-16,20MICRONS.NS,NaN,138.484528,138.484528,138.484528,138.484528,40521.0,143.386910,143.844977,144.007918,144.091431,NaN,NaN,20MICRONS
3,2023-10-17,20MICRONS.NS,NaN,141.206696,141.206696,138.484519,141.206696,34376.0,143.179271,143.741515,143.952448,144.062728,NaN,NaN,20MICRONS
4,2023-10-18,20MICRONS.NS,NaN,138.385544,143.136982,138.385544,138.385544,20591.0,142.722725,143.531477,143.842212,144.006238,NaN,NaN,20MICRONS


,Symbol,Date,AD_Slope,Above_20EMA_%,Avg_Volume_Spike,RSI,Accumulating,20ema,50ema,200ema,Signal_Price,7dv,30dv,Signal_date,Updated_Signal_Price
0,GLOBE,2025-11-07,106228.7604,80.0,1.43,68.49,True,2.403263,2.319979,2.695159,2.510000,2.303447e+06,1.105821e+06,2025-11-07,NaN
1,RBZJEWEL,2025-11-07,1155.2315,80.0,1.41,65.00,True,146.405586,142.920478,147.095673,156.350006,3.652081e+05,1.561898e+05,2025-11-07,NaN
2,ASHOKA,2025-11-07,124864.8468,90.0,1.30,58.05,True,196.244525,193.496737,201.960307,198.919998,1.577122e+06,8.100797e+05,2025-11-07,NaN
3,ASTRAL,2025-11-07,267446.4446,95.0,1.26,73.01,True,1465.859318,1443.114895,1487.197912,1557.300049,1.247884e+06,7.025423e+05,2025-11-07,NaN
4,REPCOHOME,2025-11-07,34139.5674,100.0,1.07,62.96,True,412.542336,396.749951,397.905690,425.600006,2.851806e+05,1.827009e+05,2025-11-07,NaN


In [36]:
sd = stock_df.copy(deep=True)
sd["Date"] = pd.to_datetime(sd["Date"])

display(sd.head())
signals_df = accum_df.copy(deep=True)
signals_df.rename(columns={"Close":"Signal_Price"}, inplace=True)
signals_df["Signal_date"] = pd.to_datetime(accum_df["Date"]) # Replace with Signal_date column later
signals_df["Updated_Signal_Price"] = np.nan
signals_df.head()

merged = pd.merge(
    signals_df,
    sd[["Symbol", "Date", "Close"]],
    left_on=["Symbol", "Signal_date"],
    right_on=["Symbol", "Date"],
    how="left"
)
merged["Updated_Signal_Price"] = merged['Close']
merged.head()


Price,Date,Ticker,Adj Close,Close,High,Low,Open,Volume,20ema,50ema,100ema,200ema,30dv,7dv,Symbol
0,2023-10-12,20MICRONS.NS,NaN,144.176346,145.166228,144.027870,144.176346,66373.0,144.176346,144.176346,144.176346,144.176346,NaN,NaN,20MICRONS
1,2023-10-13,20MICRONS.NS,NaN,141.305695,144.077368,141.305695,141.305695,29996.0,143.902950,144.063771,144.119501,144.147782,NaN,NaN,20MICRONS
2,2023-10-16,20MICRONS.NS,NaN,138.484528,138.484528,138.484528,138.484528,40521.0,143.386910,143.844977,144.007918,144.091431,NaN,NaN,20MICRONS
3,2023-10-17,20MICRONS.NS,NaN,141.206696,141.206696,138.484519,141.206696,34376.0,143.179271,143.741515,143.952448,144.062728,NaN,NaN,20MICRONS
4,2023-10-18,20MICRONS.NS,NaN,138.385544,143.136982,138.385544,138.385544,20591.0,142.722725,143.531477,143.842212,144.006238,NaN,NaN,20MICRONS


,Symbol,Date_x,AD_Slope,Above_20EMA_%,Avg_Volume_Spike,RSI,Accumulating,20ema,50ema,200ema,Signal_Price,7dv,30dv,Signal_date,Updated_Signal_Price,Date_y,Close
0,GLOBE,2025-11-07,106228.7604,80.0,1.43,68.49,True,2.403263,2.319979,2.695159,2.510000,2.303447e+06,1.105821e+06,2025-11-07,2.510000,2025-11-07,2.510000
1,RBZJEWEL,2025-11-07,1155.2315,80.0,1.41,65.00,True,146.405586,142.920478,147.095673,156.350006,3.652081e+05,1.561898e+05,2025-11-07,156.350006,2025-11-07,156.350006
2,ASHOKA,2025-11-07,124864.8468,90.0,1.30,58.05,True,196.244525,193.496737,201.960307,198.919998,1.577122e+06,8.100797e+05,2025-11-07,198.919998,2025-11-07,198.919998
3,ASTRAL,2025-11-07,267446.4446,95.0,1.26,73.01,True,1465.859318,1443.114895,1487.197912,1557.300049,1.247884e+06,7.025423e+05,2025-11-07,1557.300049,2025-11-07,1557.300049
4,REPCOHOME,2025-11-07,34139.5674,100.0,1.07,62.96,True,412.542336,396.749951,397.905690,425.600006,2.851806e+05,1.827009e+05,2025-11-07,425.600006,2025-11-07,425.600006


In [55]:
sd = stock_df.copy(deep=True)
sd["Date"] = pd.to_datetime(sd["Date"])
print(sd.shape)
display(sd.head())

conn = sqlite3.connect(db_path)
existing = pd.read_sql_query("SELECT * FROM SIGNAL_RETURNS", conn)
print(existing.shape)
existing.head()



(1029075, 15)


Price,Date,Ticker,Adj Close,Close,High,Low,Open,Volume,20ema,50ema,100ema,200ema,30dv,7dv,Symbol
0,2023-10-12,20MICRONS.NS,NaN,144.176346,145.166228,144.027870,144.176346,66373.0,144.176346,144.176346,144.176346,144.176346,NaN,NaN,20MICRONS
1,2023-10-13,20MICRONS.NS,NaN,141.305695,144.077368,141.305695,141.305695,29996.0,143.902950,144.063771,144.119501,144.147782,NaN,NaN,20MICRONS
2,2023-10-16,20MICRONS.NS,NaN,138.484528,138.484528,138.484528,138.484528,40521.0,143.386910,143.844977,144.007918,144.091431,NaN,NaN,20MICRONS
3,2023-10-17,20MICRONS.NS,NaN,141.206696,141.206696,138.484519,141.206696,34376.0,143.179271,143.741515,143.952448,144.062728,NaN,NaN,20MICRONS
4,2023-10-18,20MICRONS.NS,NaN,138.385544,143.136982,138.385544,138.385544,20591.0,142.722725,143.531477,143.842212,144.006238,NaN,NaN,20MICRONS


(2173, 34)


,Symbol,Signal_Type,Signal_date,Signal_Price,Signal_Rank,ret_1w,ret_1w_price,ret_1w_date,ret_2w,ret_2w_price,ret_2w_date,ret_1m,ret_1m_price,ret_1m_date,ret_3m,ret_3m_price,ret_3m_date,ret_6m,ret_6m_price,ret_6m_date,ret_1y,ret_1y_price,ret_1y_date,ret_sinceSignal,current_price,current_date,ret_1w_perc,ret_2w_perc,ret_1m_perc,ret_3m_perc,ret_6m_perc,ret_1y_perc,ret_sinceSignal_perc,Updated_Signal_Price
0,20MICRONS,Golden cross,2025-07-17 00:00:00,254.0,4,6.43,270.329987,2025-07-24 00:00:00,-7.69,234.470001,2025-07-31 00:00:00,-10.47,227.399994,2025-08-18 00:00:00,-14.98,215.960007,2025-10-15 00:00:00,NaN,NaN,None,NaN,NaN,None,-18.21,207.740005,2025-10-30 00:00:00,6.43,-7.69,-10.47,-14.98,None,None,-18.21,None
1,20MICRONS,Golden cross,2025-07-16 00:00:00,259.0,3,6.86,276.779999,2025-07-23 00:00:00,-5.46,244.850006,2025-07-30 00:00:00,-12.20,227.399994,2025-08-18 00:00:00,-17.18,214.500000,2025-10-14 00:00:00,NaN,NaN,None,NaN,NaN,None,-19.79,207.740005,2025-10-30 00:00:00,6.86,-5.46,-12.2,-17.18,None,None,-19.79,None
2,20MICRONS,Golden cross,2025-06-17 00:00:00,237.0,6,-4.12,227.243713,2025-06-24 00:00:00,-2.58,230.897217,2025-07-01 00:00:00,7.12,253.863495,2025-07-17 00:00:00,-4.95,225.270004,2025-09-15 00:00:00,NaN,NaN,None,NaN,NaN,None,-12.35,207.740005,2025-10-30 00:00:00,-4.12,-2.58,7.12,-4.95,None,None,-12.35,None
3,360ONE,Golden cross,2025-06-12 00:00:00,1081.0,5,2.57,1108.760864,2025-06-19 00:00:00,7.21,1158.955444,2025-06-26 00:00:00,9.36,1182.187256,2025-07-14 00:00:00,-0.78,1072.544922,2025-09-10 00:00:00,NaN,NaN,None,NaN,NaN,None,3.57,1119.599976,2025-10-30 00:00:00,2.57,7.21,9.36,-0.78,None,None,3.57,None
4,3MINDIA,Accumulation Signal,2025-08-07 00:00:00,32000.0,4,-5.16,30350.000000,2025-08-14 00:00:00,-3.52,30875.000000,2025-08-21 00:00:00,-3.64,30835.000000,2025-09-08 00:00:00,NaN,NaN,None,NaN,NaN,None,NaN,NaN,None,-7.75,29520.000000,2025-10-31 00:00:00,-5.16,-3.52,-3.64,None,None,None,-7.75,None


In [69]:
existing["Signal_date"] = pd.to_datetime(existing["Signal_date"]) # Replace with Signal_date column later
existing["Updated_Signal_Price"] = np.nan
existing.head()

merged = pd.merge(
    existing,
    sd[["Symbol", "Date", "Close"]],
    left_on=["Symbol", "Signal_date"],
    right_on=["Symbol", "Date"],
    how="left"
)
merged["Updated_Signal_Price"] = merged['Close'].round()
merged.drop(columns=["Date", "Close"], inplace=True)
merged.head()
print(merged.shape)
merged.head()

(2173, 34)


,Symbol,Signal_Type,Signal_date,Signal_Price,Signal_Rank,ret_1w,ret_1w_price,ret_1w_date,ret_2w,ret_2w_price,ret_2w_date,ret_1m,ret_1m_price,ret_1m_date,ret_3m,ret_3m_price,ret_3m_date,ret_6m,ret_6m_price,ret_6m_date,ret_1y,ret_1y_price,ret_1y_date,ret_sinceSignal,current_price,current_date,ret_1w_perc,ret_2w_perc,ret_1m_perc,ret_3m_perc,ret_6m_perc,ret_1y_perc,ret_sinceSignal_perc,Updated_Signal_Price
0,20MICRONS,Golden cross,2025-07-17,254.0,4,6.43,270.329987,2025-07-24 00:00:00,-7.69,234.470001,2025-07-31 00:00:00,-10.47,227.399994,2025-08-18 00:00:00,-14.98,215.960007,2025-10-15 00:00:00,NaN,NaN,None,NaN,NaN,None,-18.21,207.740005,2025-10-30 00:00:00,6.43,-7.69,-10.47,-14.98,None,None,-18.21,254.0
1,20MICRONS,Golden cross,2025-07-16,259.0,3,6.86,276.779999,2025-07-23 00:00:00,-5.46,244.850006,2025-07-30 00:00:00,-12.20,227.399994,2025-08-18 00:00:00,-17.18,214.500000,2025-10-14 00:00:00,NaN,NaN,None,NaN,NaN,None,-19.79,207.740005,2025-10-30 00:00:00,6.86,-5.46,-12.2,-17.18,None,None,-19.79,259.0
2,20MICRONS,Golden cross,2025-06-17,237.0,6,-4.12,227.243713,2025-06-24 00:00:00,-2.58,230.897217,2025-07-01 00:00:00,7.12,253.863495,2025-07-17 00:00:00,-4.95,225.270004,2025-09-15 00:00:00,NaN,NaN,None,NaN,NaN,None,-12.35,207.740005,2025-10-30 00:00:00,-4.12,-2.58,7.12,-4.95,None,None,-12.35,236.0
3,360ONE,Golden cross,2025-06-12,1081.0,5,2.57,1108.760864,2025-06-19 00:00:00,7.21,1158.955444,2025-06-26 00:00:00,9.36,1182.187256,2025-07-14 00:00:00,-0.78,1072.544922,2025-09-10 00:00:00,NaN,NaN,None,NaN,NaN,None,3.57,1119.599976,2025-10-30 00:00:00,2.57,7.21,9.36,-0.78,None,None,3.57,1076.0
4,3MINDIA,Accumulation Signal,2025-08-07,32000.0,4,-5.16,30350.000000,2025-08-14 00:00:00,-3.52,30875.000000,2025-08-21 00:00:00,-3.64,30835.000000,2025-09-08 00:00:00,NaN,NaN,None,NaN,NaN,None,NaN,NaN,None,-7.75,29520.000000,2025-10-31 00:00:00,-5.16,-3.52,-3.64,None,None,None,-7.75,32000.0


In [58]:
# print(existing.columns)
# print(merged.columns)

In [70]:
merged.to_csv("D:/Work/Projects/Stock_Market/Stock_Market/Stock_signals/Stock-Signals/data/merged_signals_updated_prices.csv", index=False)

In [96]:
existing.shape, stock_df.shape

((2173, 34), (1029075, 15))

In [91]:
stock_df.columns

Index(['Date', 'Ticker', 'Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume',
       '20ema', '50ema', '100ema', '200ema', '30dv', '7dv', 'Symbol'],
      dtype='object', name='Price')

In [118]:

HORIZONS = {
    "ret_1w": 7,
    "ret_2w": 14,
    "ret_1m": 30,
    "ret_3m": 90,
    "ret_6m": 180,
    "ret_1y": 365,
}

def merge_signals_with_prices(signals_df, stock_df):
    """Join signal data with stock prices to get post-signal price evolution."""

    print("Added by Deepak- to update signal price based on corporate actions, current merge shape",signals_df.shape)

    existing_df = signals_df.copy(deep=True)   
    existing_df["Signal_date"] = pd.to_datetime(existing_df["Signal_date"]) # Replace with Signal_date column later
    existing_df["Updated_Signal_Price"] = np.nan
    existing_df.head()

    merged = pd.merge(
        existing_df,
        stock_df[["Symbol", "Date", "Close"]],
        left_on=["Symbol", "Signal_date"],
        right_on=["Symbol", "Date"],
        how="left"
    )
    
    merged["Updated_Signal_Price"] = merged['Close'].round()
    merged.drop(columns=["Date", "Close"], inplace=True)
    print(merged.columns)
    print(merged.shape)
    print(merged.head())


    format_stock_df = stock_df[["Symbol", "Date", "Close"]].copy()
    format_stock_df["Date"] = pd.to_datetime(format_stock_df["Date"])

    print("Total signals",merged.shape)
    print("current data pull",format_stock_df.shape)


    merged2 = pd.merge(merged, format_stock_df, on="Symbol", how="inner")
    merged2 = merged2[["Symbol", "Signal_date", "Signal_Type", "Signal_Price","Updated_Signal_Price","Signal_Rank", "Date", "Close"]]
    merged2 = merged2.dropna()
    
    merged2["Date"] = pd.to_datetime(merged2["Date"])
    merged2 = merged2[merged2["Date"] >= pd.to_datetime(merged2["Signal_date"])]

    merged2 = merged2.sort_values(
        by=["Symbol", "Signal_Type", "Signal_date", "Date"],
        ascending=[True, True, False, False],
    )
    merged2.rename(columns={"Close": "Price"}, inplace=True)

    print(f"✅ Total signals after merging: {merged2.shape[0]}")

    

    return merged2

def compute_returns(group):
    """Compute returns for a single (Symbol, Signal_Type, Signal_date, Signal_Price,Updated_Signal_Price, Signal_Rank) group."""
    group = group.sort_values("Date").reset_index(drop=True)

    # ensure numeric columns before calculation
    group["Price"] = pd.to_numeric(group["Price"], errors="coerce")
    group["Signal_Price"] = pd.to_numeric(group["Signal_Price"], errors="coerce")

    sig_date = group.at[0, "Signal_date"]
    sig_price = group.at[0, "Updated_Signal_Price"]

    out = {
        "Symbol": group.at[0, "Symbol"],
        "Signal_Type": group.at[0, "Signal_Type"],
        "Signal_date": sig_date,
        "Signal_Price": group.at[0, "Signal_Price"],
        "Updated_Signal_Price": sig_price ,
        "Signal_Rank": group.at[0, "Signal_Rank"],
    }

    for name, days in HORIZONS.items():
        target = sig_date + pd.Timedelta(days=days)
        sel = group[group["Date"] >= target]

        if not sel.empty:
            p = pd.to_numeric(sel.iloc[0]["Price"], errors="coerce")
            d = sel.iloc[0]["Date"]

            if pd.notna(p) and pd.notna(sig_price) and sig_price != 0:
                out[name] = round((p / sig_price - 1) * 100, 2)
            else:
                out[name] = pd.NA

            out[f"{name}_price"] = p
            out[f"{name}_date"] = d
        else:
            out[name] = pd.NA
            out[f"{name}_price"] = pd.NA
            out[f"{name}_date"] = pd.NaT

    last_row = group.iloc[-1]
    last_price = pd.to_numeric(last_row["Price"], errors="coerce")
    if pd.notna(last_price) and pd.notna(sig_price) and sig_price != 0:
        out["ret_sinceSignal"] = round((last_price / sig_price - 1) * 100, 2)
    else:
        out["ret_sinceSignal"] = pd.NA

    out["current_price"] = last_price
    out["current_date"] = last_row["Date"]

    return pd.Series(out)

def calculate_returns(merged_df):
    """Apply return computation across all signal groups."""
    merged_df["Signal_date"] = pd.to_datetime(merged_df["Signal_date"])
    merged_df["Date"] = pd.to_datetime(merged_df["Date"])

    returns_df = (
        merged_df.groupby(["Symbol", "Signal_Type", "Signal_date", "Signal_Price","Updated_Signal_Price","Signal_Rank"], sort=False)
        .apply(compute_returns)
        .reset_index(drop=True)
    )

    # Convert numeric columns to ensure no text values
    ret_cols = [c for c in returns_df.columns if c.startswith("ret_") and not c.endswith("_price") and not c.endswith("_date")]
    for c in ret_cols:
        returns_df[c] = pd.to_numeric(returns_df[c], errors="coerce").round(2)

    price_cols = [c for c in returns_df.columns if c.endswith("_price")]
    for c in price_cols:
        returns_df[c] = pd.to_numeric(returns_df[c], errors="coerce")

    returns_df["Updated_Signal_Price"] = pd.to_numeric(returns_df["Updated_Signal_Price"], errors="coerce")

    # Calculate percentage returns based on *_price and Updated_Signal_Price
    for horizon in HORIZONS.keys():
        returns_df[f"{horizon}_perc"] = returns_df.apply(
            lambda row: round((row[f"{horizon}_price"] / row["Updated_Signal_Price"] - 1) * 100, 2)
            if pd.notna(row[f"{horizon}_price"]) and pd.notna(row["Updated_Signal_Price"]) and row["Updated_Signal_Price"] != 0
            else pd.NA,
            axis=1
        )

    # ret_sinceSignal_perc
    returns_df["ret_sinceSignal_perc"] = returns_df.apply(
        lambda row: round((row["current_price"] / row["Updated_Signal_Price"] - 1) * 100, 2)
        if pd.notna(row["current_price"]) and pd.notna(row["Updated_Signal_Price"]) and row["Updated_Signal_Price"] != 0
        else pd.NA,
        axis=1
    )


    return returns_df

merged_df = merge_signals_with_prices(existing, stock_df)

returns_df = calculate_returns(merged_df)

returns_df.columns

Added by Deepak- to update signal price based on corporate actions, current merge shape (2173, 34)
Index(['Symbol', 'Signal_Type', 'Signal_date', 'Signal_Price', 'Signal_Rank',
       'ret_1w', 'ret_1w_price', 'ret_1w_date', 'ret_2w', 'ret_2w_price',
       'ret_2w_date', 'ret_1m', 'ret_1m_price', 'ret_1m_date', 'ret_3m',
       'ret_3m_price', 'ret_3m_date', 'ret_6m', 'ret_6m_price', 'ret_6m_date',
       'ret_1y', 'ret_1y_price', 'ret_1y_date', 'ret_sinceSignal',
       'current_price', 'current_date', 'ret_1w_perc', 'ret_2w_perc',
       'ret_1m_perc', 'ret_3m_perc', 'ret_6m_perc', 'ret_1y_perc',
       'ret_sinceSignal_perc', 'Updated_Signal_Price'],
      dtype='object')
(2173, 34)
      Symbol          Signal_Type Signal_date  Signal_Price  Signal_Rank  \
0  20MICRONS         Golden cross  2025-07-17         254.0            4   
1  20MICRONS         Golden cross  2025-07-16         259.0            3   
2  20MICRONS         Golden cross  2025-06-17         237.0            6   


Index(['Symbol', 'Signal_Type', 'Signal_date', 'Signal_Price',
       'Updated_Signal_Price', 'Signal_Rank', 'ret_1w', 'ret_1w_price',
       'ret_1w_date', 'ret_2w', 'ret_2w_price', 'ret_2w_date', 'ret_1m',
       'ret_1m_price', 'ret_1m_date', 'ret_3m', 'ret_3m_price', 'ret_3m_date',
       'ret_6m', 'ret_6m_price', 'ret_6m_date', 'ret_1y', 'ret_1y_price',
       'ret_1y_date', 'ret_sinceSignal', 'current_price', 'current_date',
       'ret_1w_perc', 'ret_2w_perc', 'ret_1m_perc', 'ret_3m_perc',
       'ret_6m_perc', 'ret_1y_perc', 'ret_sinceSignal_perc'],
      dtype='object')

In [119]:
print(merged_df.columns)
print(returns_df.columns)
returns_df[returns_df['Symbol'] == 'COFORGE']

Index(['Symbol', 'Signal_date', 'Signal_Type', 'Signal_Price',
       'Updated_Signal_Price', 'Signal_Rank', 'Date', 'Price'],
      dtype='object')
Index(['Symbol', 'Signal_Type', 'Signal_date', 'Signal_Price',
       'Updated_Signal_Price', 'Signal_Rank', 'ret_1w', 'ret_1w_price',
       'ret_1w_date', 'ret_2w', 'ret_2w_price', 'ret_2w_date', 'ret_1m',
       'ret_1m_price', 'ret_1m_date', 'ret_3m', 'ret_3m_price', 'ret_3m_date',
       'ret_6m', 'ret_6m_price', 'ret_6m_date', 'ret_1y', 'ret_1y_price',
       'ret_1y_date', 'ret_sinceSignal', 'current_price', 'current_date',
       'ret_1w_perc', 'ret_2w_perc', 'ret_1m_perc', 'ret_3m_perc',
       'ret_6m_perc', 'ret_1y_perc', 'ret_sinceSignal_perc'],
      dtype='object')


,Symbol,Signal_Type,Signal_date,Signal_Price,Updated_Signal_Price,Signal_Rank,ret_1w,ret_1w_price,ret_1w_date,ret_2w,ret_2w_price,ret_2w_date,ret_1m,ret_1m_price,ret_1m_date,ret_3m,ret_3m_price,ret_3m_date,ret_6m,ret_6m_price,ret_6m_date,ret_1y,ret_1y_price,ret_1y_date,ret_sinceSignal,current_price,current_date,ret_1w_perc,ret_2w_perc,ret_1m_perc,ret_3m_perc,ret_6m_perc,ret_1y_perc,ret_sinceSignal_perc
389,COFORGE,Death cross,2024-05-08,4410.0,865.0,1,5.14,909.455872,2024-05-15,11.87,967.669556,2024-05-22,23.73,1070.292358,2024-06-07,34.73,1165.431519,2024-08-06,72.82,1494.935791,2024-11-04,78.05,1540.147339,2025-05-08,101.18,1740.199951,2025-11-07,5.14,11.87,23.73,34.73,72.82,78.05,101.18


In [120]:
returns_df.to_csv("D:/Work/Projects/Stock_Market/Stock_Market/Stock_signals/Stock-Signals/data/returns_df.csv", index=False)

In [ ]:





merged.drop(columns=["Date", "Close"], inplace=True)

print(f"🔄 Corporate action adjustments applied to {sum(merged['Updated_Signal_Price'] != merged['Signal_Price'])} signals")